In [33]:
lst = list(range(10))

lst
lst.append('black sheep')

In [34]:
final = all(map(lambda v: isinstance(v, int), lst))
final

False

In [37]:

# new attempt

n = 'lion'
type_list = [int, tuple, float]

# if n is not of type in some list
boo = not any(map(lambda t: isinstance(n, t), type_list))
boo

True